In [21]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import cross_validate

In [2]:
data = pd.read_csv('data/unconv.csv')
data.head()

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
0,1,12.08,2.92,2.80,81.40,1.16,2.31,4165.196191
1,2,12.38,3.53,3.22,46.17,0.89,1.88,3561.146205
2,3,14.02,2.59,4.01,72.80,0.89,2.72,4284.348574
3,4,17.67,6.75,2.63,39.81,1.08,1.88,5098.680869
4,5,17.52,4.57,3.18,10.94,1.51,1.90,3406.132832


In [3]:
np.round(data.corr(), 2)

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
Well,1.00,0.07,0.08,0.04,-0.08,0.02,-0.01,0.03
Por,0.07,1.00,0.76,-0.46,-0.22,0.71,0.11,0.86
Perm,0.08,0.76,1.00,-0.24,-0.12,0.47,0.05,0.73
AI,0.04,-0.46,-0.24,1.00,0.13,-0.53,0.50,-0.39
Brittle,-0.08,-0.22,-0.12,0.13,1.00,-0.21,0.32,0.24
TOC,0.02,0.71,0.47,-0.53,-0.21,1.00,0.30,0.65
VR,-0.01,0.11,0.05,0.50,0.32,0.30,1.00,0.32
Prod,0.03,0.86,0.73,-0.39,0.24,0.65,0.32,1.00


In [4]:
np.linalg.matrix_rank(data.corr())

8

In [5]:
np.round(np.linalg.det(data.corr()), 4)

0.0007

In [6]:
X = data.drop(['Prod'], axis=1)
y = data['Prod']
X = np.column_stack((np.ones(X.shape[0]), X))
X.shape

(200, 8)

In [7]:
w = np.linalg.inv(X.T@X)@X.T@y
w

array([-1.23230803e+03,  5.07003631e-02,  2.30179140e+02,  1.16239006e+02,
       -3.65202301e+02,  2.49943700e+01, -7.84009294e+01,  7.85259815e+02])

In [8]:
X1 = np.array([1, 106, 15.32, 3.71, 3.29, 55.99, 1.35, 2.42])
y1 = 4748.315024

In [9]:
y1_pred = np.dot(X1, w)
y1_pred
y1 - y1_pred

25.250970292820057

In [10]:
y_pred = np.dot(X, w)

metrics.mean_absolute_percentage_error(y, y_pred) * 100

3.6279468451339723

In [11]:
X_new = data.drop(['Prod', 'Well', 'Perm', 'TOC'], axis=1)
y = data['Prod']
X_new = np.column_stack((np.ones(X_new.shape[0]), X_new))
X_new.shape

(200, 5)

In [12]:
w_new = np.linalg.inv(X_new.T@X_new)@X_new.T@y
w_new

array([-1835.44646069,   293.03624565,  -200.03091206,    27.64098209,
         517.40272597])

In [13]:
y_pred = np.dot(X_new, w_new)

metrics.mean_absolute_percentage_error(y, y_pred) * 100

4.04413842043617

In [28]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_new)

print(f'Количество полиноминальных признаков: {X_poly.shape}')

lr = LinearRegression()
lr.fit(X_poly, y)

cv_results = cross_validate(lr, X_poly, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))	

Количество полиноминальных признаков: (200, 20)
MAPE на тренировочных фолдах: 2.37 %
MAPE на валидационных фолдах: 2.63 %
